#Ajuste con MCMC usando emcee

Se muestra el desarrollo para un pixel. Para los 100 pixeles es sólo hacer un for para varios pixeles pero siempre ejecutando el procedimiento que aquí se muestra. Cada uno de los archivos se escribirían las constantes que se obtienen en cada caso. No consideramos necesario hacer el desarrollo completo del problema para emcee ya que se muestra en este notebook el proceso esencial. El resto sería escribir un código que escriba las constantes en un archivo pero este proceso ya se ha hecho en el punto 2a.

In [2]:
%pylab inline
import emcee
import pyfits

Populating the interactive namespace from numpy and matplotlib


In [25]:
sdo = pyfits.open('./data/hmi.m_45s.magnetogram.subregion_x1y1.fits')
time = np.loadtxt("./data/time_series.csv")
data = sdo[0].data
campo = data[:, 135, 235]

##Definición de modelos

In [5]:
def linear_model(x_obs, a, b):
    return x_obs*b + a

In [9]:
def gauss_model(t,c,d,sigma,mu):
	return c + d*t + (1/(sigma*np.sqrt(2*math.pi)))*np.exp(-0.5*((t-mu)/sigma)**2)

In [4]:
def step_model(t, f, g, h, n, t_0):
    return f + g*t + h*(1+(2/math.pi)*np.arctan(n*(t-t_0)))

##Definicion de funciones para usar emcee

###Lnprior

In [8]:
def linear_lnprior(constantes):
    a, b = constantes
    if -200 < a < 200 and -1 < b < 1:
        return 0.0
    return -inf
    

In [10]:
def gauss_lnprior(constantes):
    c, d, sigma, mu= constantes
    if -200 < c < 200 and -1 < d < 1 and 30 < sigma < 50 and 80 < mu < 120 :
        return 0.0
    return -inf

In [11]:
def step_lnprior(constantes):
    f, g, h, n, t_0 = constantes
    if -200 < f < 200 and -1 < g < 1 and -100 < h < 100 and 0 < n < 100 and 80 < t_0 <350:
        return 0.0
    return -inf

###Lnlike

In [7]:
def linear_lnlike(constantes, time, y_obs):
    a, b = constantes
    y_model = linear_model(time, a, b)
    chi_squared = (1.0/2.0)*sum((y_obs-y_model)**2)
    return -chi_squared
    

In [12]:
def gauss_lnlike(constantes, time, y_obs):
    c, d, sigma, mu= constantes
    y_model = gauss_model(time, c, d, sigma, mu)
    chi_squared = (1.0/2.0)*sum((y_obs-y_model)**2)
    return -chi_squared

In [103]:
def step_lnlike(constantes, time, y_obs):
    f, g, h, n, t_0 = constantes
    y_model = step_model(time, f, g, h, n, t_0)
    chi_squared = (1.0/2.0)*sum((y_obs-y_model)**2)
    return -chi_squared

###Lnprob

In [35]:
def linear_lnprob(constantes, time, y_obs):
    lp = linear_lnprior(constantes)
    if not isfinite(lp):
        return -inf
    return lp +  linear_lnlike(constantes, time, y_obs)

In [36]:
def gauss_lnprob(constantes, time, y_obs):
    lp = gauss_lnprior(constantes)
    if not isfinite(lp):
        return -inf
    return lp +  gauss_lnlike(constantes, time, y_obs)

In [34]:
def step_lnprob(constantes, time, y_obs):
    lp = step_lnprior(constantes)
    if not isfinite(lp):
        return -inf
    return lp +  step_lnlike(constantes, time, y_obs)

##Inicialización de constantes

Se inicializan las constantes con un "first guess" de su valor

In [19]:
a = 0.1
c = 0.1
f = 0.1
b = 0.1
d = 0.1
g = 0.1
h = 0.1
n = 50
t_0 = 210
mu = 100
sigma = 40

##Ejecución de emcee

El archivo que se genera en cada caso contiene los valores de las constantes en toda la caminata. Los valores que se imprimen como constante_fit son el valor que más se repite en la caminata con el número de veces que se repite

###Ajuste lineal

In [63]:
y_lineal = linear_model(time, a, b)
first_guess  = [a, b]

In [64]:
ndim = 2
nwalkers = 10
n_iterations = 2000

In [65]:
p_linear = [first_guess + random.randn(ndim) for i in range(nwalkers)]

In [66]:
linear_sampler = emcee.EnsembleSampler(nwalkers, ndim, linear_lnprob, args=(time, campo))

In [67]:
linear_sampler.run_mcmc(p_linear, n_iterations, rstate0= random.get_state())

(array([[ 21.93327543,   0.04875298],
        [ 21.82751421,   0.04944198],
        [ 22.01817911,   0.04857453],
        [ 21.98074156,   0.04855641],
        [ 22.20078193,   0.04818652],
        [ 21.90485848,   0.048694  ],
        [ 22.06767141,   0.04837974],
        [ 22.04689187,   0.04835122],
        [ 22.16432599,   0.04849268],
        [ 22.18548731,   0.04768525]]),
 array([-618.0201719 , -618.42741443, -617.9219034 , -618.02017086,
        -619.08892479, -618.48649391, -617.98002195, -617.99953867,
        -619.8502131 , -618.73386653]),
 ('MT19937', array([4116241185,  882349400, 1360736046, 3729887055, 4042090810,
          512207930, 2798034770, 4036254870, 2793671886, 3602468169,
         3617443091, 3502322692, 1693137450, 1247798892,   73420390,
          223332348,  371302239, 1093262438,  140677130, 1633333620,
         1466959873, 2797594340, 1663763529,  215811042,  468893196,
          821843128, 1911456008, 1828053190, 3821581904, 4081704786,
          2752913

In [69]:
linear_samples = linear_sampler.flatchain
savetxt('./emcee_Samples/linear_sampler.dat', linear_samples, delimiter = ',')

In [90]:
a_list = linear_samples[:, 0].tolist()
a_count = Counter(a_list)
a_fit = a_count.most_common(1) 
print a_fit

[(0.676146542632257, 13)]


In [91]:
b_list = linear_samples[:, 1].tolist()
b_count = Counter(b_list)
b_fit = b_count.most_common(1) 
print b_fit

[(0.12718387447343105, 13)]


###Ajuste gaussiano

In [70]:
y_gauss = gauss_model(time, c, d, sigma, mu)
first_guess  = [c, d, sigma, mu]

In [71]:
ndim = 4
nwalkers = 10
n_iterations = 2000

In [72]:
p_gauss = [first_guess + random.randn(ndim) for i in range(nwalkers)]

In [58]:
gauss_sampler = emcee.EnsembleSampler(nwalkers, ndim, gauss_lnprob, args=(time, campo))

In [73]:
gauss_sampler.run_mcmc(p_gauss, n_iterations, rstate0= random.get_state())

(array([[  2.17693810e+01,   4.97000229e-02,   4.57107494e+01,
           1.19093449e+02],
        [  2.17759582e+01,   4.99461732e-02,   3.34434347e+01,
           1.18105142e+02],
        [  2.16689898e+01,   5.00316979e-02,   4.26783195e+01,
           1.10482085e+02],
        [  2.19777358e+01,   4.88930257e-02,   3.54905306e+01,
           8.92097331e+01],
        [  2.24135237e+01,   4.74428618e-02,   4.68754986e+01,
           8.38533931e+01],
        [  2.16349799e+01,   4.98065209e-02,   4.56784962e+01,
           1.05384979e+02],
        [  2.20847178e+01,   4.82368629e-02,   4.96023241e+01,
           8.03298411e+01],
        [  2.20104845e+01,   4.82690149e-02,   4.44755442e+01,
           9.63077232e+01],
        [  2.21046945e+01,   4.81002224e-02,   3.13089650e+01,
           9.56734092e+01],
        [  2.16942433e+01,   4.96342285e-02,   3.65710642e+01,
           9.35434493e+01]]),
 array([-618.72797374, -620.06031772, -619.252283  , -618.25648138,
        -620.9812661

In [74]:
gauss_samples = gauss_sampler.flatchain
savetxt('./emcee_Samples/gauss_sampler.dat', gauss_samples, delimiter = ',')

In [94]:
c_list = gauss_samples[:, 0].tolist()
c_count = Counter(c_list)
c_fit = c_count.most_common(1) 
print c_fit

[(-0.7371313631687691, 2000)]


In [95]:
d_list = gauss_samples[:, 1].tolist()
d_count = Counter(d_list)
d_fit = d_count.most_common(1) 
print d_fit

[(2.3887857234210177, 2000)]


In [96]:
sigma_list = gauss_samples[:, 2].tolist()
sigma_count = Counter(sigma_list)
sigma_fit = sigma_count.most_common(1) 
print sigma_fit

[(40.66080334310102, 2000)]


In [97]:
mu_list = gauss_samples[:, 3].tolist()
mu_count = Counter(mu_list)
mu_fit = mu_count.most_common(1) 
print mu_fit

[(99.94092801232333, 2000)]


###Ajuste de paso

In [104]:
y_step = step_model(time, f, g, h, n, t_0)
first_guess  = [f, g, h, n, t_0]

In [105]:
ndim = 5
nwalkers = 10
n_iterations = 2000

In [106]:
p_step = [first_guess + random.randn(ndim) for i in range(nwalkers)]

In [107]:
step_sampler = emcee.EnsembleSampler(nwalkers, ndim, step_lnprob, args=(time, campo))

In [108]:
step_sampler.run_mcmc(p_step, n_iterations, rstate0= random.get_state())

(array([[  2.17350375e+01,   5.33665704e-02,  -5.98286516e-01,
           7.41081172e+01,   2.10625671e+02],
        [  2.16044556e+01,   5.51598640e-02,  -7.27041721e-01,
           3.32318665e+01,   2.09919943e+02],
        [  2.17287829e+01,   5.38280112e-02,  -6.57491022e-01,
           1.29812258e+01,   2.08951947e+02],
        [  2.19079897e+01,   5.25059758e-02,  -6.06206093e-01,
           8.67567713e+01,   2.09854415e+02],
        [  2.12291290e+01,   5.66370480e-02,  -8.33027222e-01,
           6.79211345e+01,   2.09572320e+02],
        [  2.16653339e+01,   5.37217185e-02,  -6.64983313e-01,
           2.01959601e+01,   2.09703247e+02],
        [  2.14557194e+01,   5.55834108e-02,  -8.31360978e-01,
           1.17220742e+01,   2.10838093e+02],
        [  2.13182104e+01,   5.47303296e-02,  -6.57926034e-01,
           9.87262626e+01,   2.08856822e+02],
        [  2.18281775e+01,   5.31143044e-02,  -6.69742778e-01,
           9.29848266e+01,   2.10035997e+02],
        [  2.193406

In [109]:
step_samples = step_sampler.flatchain
savetxt('./emcee_Samples/gauss_sampler.dat', gauss_samples, delimiter = ',')

In [110]:
f_list = step_samples[:, 0].tolist()
f_count = Counter(f_list)
f_fit = f_count.most_common(1) 
print f_fit

[(22.032488676864066, 22)]


In [111]:
g_list = step_samples[:, 1].tolist()
g_count = Counter(g_list)
g_fit = g_count.most_common(1) 
print g_fit

[(0.05291860863739551, 22)]


In [112]:
h_list = step_samples[:, 2].tolist()
h_count = Counter(h_list)
h_fit = h_count.most_common(1) 
print h_fit

[(-0.7303862585450296, 22)]


In [113]:
n_list = step_samples[:, 3].tolist()
n_count = Counter(n_list)
n_fit = n_count.most_common(1) 
print n_fit

[(99.83810083296488, 22)]


In [114]:
t0_list = step_samples[:, 4].tolist()
t0_count = Counter(t0_list)
t0_fit = t0_count.most_common(1) 
print t0_fit

[(209.6467837714025, 22)]
